In [2]:
import pandas as pd
from pprint import pprint

csv = pd.read_csv("raw/anime_cleaned.csv", usecols = ["studio", "anime_id", "score", "genre"])
studios = set(csv["studio"])
genres = {'Comedy','Action','Fantasy','Adventure','Drama'}
score = {9.0, 8.0,7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0, 0.0}

In [3]:
studio_dic = dict()
for s in studios:
    
    studio_dic[s] = dict()
    studio_dic[s]["length"] = len(csv[csv["studio"].str.contains(s)])
    for genre in genres:
        studio_dic[s][genre] = list(csv[csv["genre"].str.contains(genre, na=False)][csv["studio"].str.contains(s)]["anime_id"])


C:\Users\Joachim\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [45]:
len(studio_dic)

711

In [4]:
def meanScorePerGenre(animes):
    scores = dict()
    for s in score:
        scores[s] = 0
    for anime in animes:
        mean = round(csv.loc[csv["anime_id"] == anime]["score"])
        if int(mean) in score:
            scores[int(mean)] += 1
    return scores

In [5]:
def initGenreDico():
    dico = dict()
    for g in genres:
        dico[g] = 0
    return dico

def initNotesDico():
    dico = dict()
    for n in score:
        dico[n] = 0
    return dico

In [15]:
a = "1.0"
float(a)

1.0

In [27]:
def RepresentsInt(s):
    try: 
        float(s)
        return True
    except ValueError:
        return False

def sortAlphab(x, y, xTo="", yTo=""):
    if RepresentsInt(xTo): 
        if x < y: return -1
        elif x==y and xTo >= yTo: return -1
        return 1
    if x <= y: return -1
    return 1
    
def checkIfSameComponent(x, y):
    if((x["type"] == "studio" and y["type"] == "studio") or
      (x["type"] == "genre" and y["type"] == "genre") or
      (x["type"] == "info" and y["type"] == "info")): 
        return sortAlphab(x["from"], y["from"])
    if((x["type"] == "link" and "-0" in x["id"] and y["type"] == "link" and "-0" in y["id"]) or
      (x["type"] == "link" and "-1" in x["id"] and y["type"] == "link" and "-1" in y["id"])):
        return sortAlphab(x["from"], y["from"], x["to"], y["to"])
    return 10

def sankeyDatasort(x, y):
    sameComp = checkIfSameComponent(x, y) 
    if (x["type"] == "studio"): return -1 if (sameComp == 10) else sameComp
    elif (x["type"] == "genre" and not(y["type"] == "studio")): return -1 if (sameComp == 10) else sameComp
    elif (x["type"] == "note" and (not(y["type"] == "studio" or y["type"] == "genre"))): return -1 if (sameComp == 10) else sameComp
    elif (x["type"] == "link" and "-0" in x["id"] and (not(y["type"] == "studio" or y["type"] == "genre" or y["type"] == "note"))): return -1 if (sameComp == 10) else sameComp
    else: return 1

In [28]:
from functools import cmp_to_key

dataAll = []

topStudios = sorted(studio_dic.items(), reverse=True, key=lambda x: x[1]["length"])[:10]

countGenre = initGenreDico()
countNotes = initNotesDico()
                    
for studio, rest in topStudios:
    dataAll.append({"from" : studio, "value": 0, "id":"none", "info": "Number of animes producted : "+str(studio_dic[studio]["length"]), "type": "studio"})
    for genre, animes in rest.items():
        if(genre != "length"):
            currId = ''.join(filter(str.isalpha, studio))[:2].join(filter(str.isalpha, studio))[-3:]+genre[:2]
            countGenre[genre] += len(animes)
            if(len(animes) != 0):
                dataAll.append({"from": studio, "to": genre, "value": len(animes), "id": currId+"-0", "type": "link"})
            scores = meanScorePerGenre(animes)
            for note, mean in scores.items():
                countNotes[note] += mean
                if(mean != 0):
                    dataAll.append({"from": genre, "to": str(note), "value": mean, "id": currId+"-1", "type": "link"})
                    
for g, count in countGenre.items():
    dataAll.append({"from" : g, "value": 0, "id":"none", "info": "Number of animes : "+str(count), "type": "genre"})
for n, count in countNotes.items():
    dataAll.append({"from" : str(n), "value": 0, "id":"none", "info": "Number of animes : "+str(count), "type": "note"})
dataAll.append({"from": "Fantasy", "to": '0.0', "value": 1, "id": "none", "type": "link"})
dataAll.append({"from": "Fantasy", "to": '1.0', "value": 1, "id": "none", "type": "link"})
dataAll = sorted(dataAll, key=cmp_to_key(sankeyDatasort))

In [29]:
import json

with open('sankey_dataset.json', 'w') as outfile:
    json.dump(dataAll, outfile)

In [29]:
topCsv = pd.read_csv("anime_cleaned.csv", usecols = ["studio", "anime_id", "favorites"])
topAnimesPerStudio = dict()

for studio, rest in topStudios:
    topAnimesPerStudio[studio] = set()
    for genre, animes in rest.items():
        if(genre != "length"):
            for anime in animes:
                topAnimesPerStudio[studio].add((anime, int(topCsv.loc[csv["anime_id"] == anime]["favorites"])))

In [30]:
pprint(topAnimesPerStudio)

{'A-1 Pictures': {(2159, 1474),
                  (3073, 0),
                  (3366, 161),
                  (3887, 9),
                  (3958, 320),
                  (3974, 316),
                  (4898, 19614),
                  (5074, 206),
                  (5234, 0),
                  (5507, 609),
                  (5821, 1),
                  (5978, 8),
                  (6151, 13),
                  (6163, 98),
                  (6380, 18),
                  (6641, 0),
                  (6702, 37733),
                  (6707, 3587),
                  (6793, 249),
                  (6956, 1964),
                  (6973, 44),
                  (6974, 177),
                  (7016, 5),
                  (7720, 150),
                  (8115, 35),
                  (8423, 3),
                  (8449, 351),
                  (8800, 3),
                  (8840, 2),
                  (9124, 4),
                  (9136, 433),
                  (9220, 1),
                  (9314, 191),

             (918, 33781),
             (929, 23),
             (930, 52),
             (1011, 18),
             (1069, 43),
             (1090, 20),
             (1091, 13),
             (1092, 58),
             (1136, 68),
             (1241, 52),
             (1248, 9),
             (1249, 27),
             (1289, 16),
             (1314, 3),
             (1381, 3),
             (1391, 40),
             (1450, 113),
             (1470, 444),
             (1471, 52),
             (1472, 21),
             (1473, 21),
             (1474, 5),
             (1477, 1),
             (1542, 17),
             (1575, 63614),
             (1576, 118),
             (1606, 514),
             (1609, 25),
             (1659, 4),
             (1694, 30),
             (1747, 62),
             (1792, 22),
             (1794, 2),
             (1795, 2),
             (1796, 16),
             (1802, 13),
             (1803, 1),
             (1848, 6),
             (1897, 6),
             (1953, 107),
   

In [43]:
pprint(dataAll)

[{'from': 'A-1 Pictures',
  'id': 'none',
  'info': 'Number of animes producted : 174',
  'type': 'studio',
  'value': 0},
 {'from': 'AIC',
  'id': 'none',
  'info': 'Number of animes producted : 161',
  'type': 'studio',
  'value': 0},
 {'from': 'J.C.Staff',
  'id': 'none',
  'info': 'Number of animes producted : 251',
  'type': 'studio',
  'value': 0},
 {'from': 'Madhouse',
  'id': 'none',
  'info': 'Number of animes producted : 277',
  'type': 'studio',
  'value': 0},
 {'from': 'Production I.G',
  'id': 'none',
  'info': 'Number of animes producted : 222',
  'type': 'studio',
  'value': 0},
 {'from': 'Studio Deen',
  'id': 'none',
  'info': 'Number of animes producted : 223',
  'type': 'studio',
  'value': 0},
 {'from': 'Studio Pierrot',
  'id': 'none',
  'info': 'Number of animes producted : 248',
  'type': 'studio',
  'value': 0},
 {'from': 'Sunrise',
  'id': 'none',
  'info': 'Number of animes producted : 291',
  'type': 'studio',
  'value': 0},
 {'from': 'TMS Entertainment',
  '

 {'from': 'Action', 'id': 'TMtAc-1', 'to': '6.0', 'type': 'link', 'value': 5},
 {'from': 'Action', 'id': 'TMtAc-1', 'to': '7.0', 'type': 'link', 'value': 32},
 {'from': 'Action', 'id': 'TMtAc-1', 'to': '8.0', 'type': 'link', 'value': 14},
 {'from': 'Drama', 'id': 'TMtDr-1', 'to': '6.0', 'type': 'link', 'value': 2},
 {'from': 'Drama', 'id': 'TMtDr-1', 'to': '7.0', 'type': 'link', 'value': 12},
 {'from': 'Drama', 'id': 'TMtDr-1', 'to': '8.0', 'type': 'link', 'value': 15},
 {'from': 'Comedy', 'id': 'TMtCo-1', 'to': '4.0', 'type': 'link', 'value': 1},
 {'from': 'Comedy', 'id': 'TMtCo-1', 'to': '5.0', 'type': 'link', 'value': 5},
 {'from': 'Comedy', 'id': 'TMtCo-1', 'to': '6.0', 'type': 'link', 'value': 17},
 {'from': 'Comedy', 'id': 'TMtCo-1', 'to': '7.0', 'type': 'link', 'value': 37},
 {'from': 'Comedy', 'id': 'TMtCo-1', 'to': '8.0', 'type': 'link', 'value': 41},
 {'from': 'Comedy', 'id': 'TMtCo-1', 'to': '9.0', 'type': 'link', 'value': 1},
 {'from': 'Adventure',
  'id': 'TMtAd-1',
  'to'